**-----------VP MWV Avg Analysis---------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 3173.0100 sec, Std Dev: 40.4465',
    'Total Time - Mean: 3237.0700 sec, Std Dev: 279.0243',
    'Total Time - Mean: 3448.3350 sec, Std Dev: 48.7833'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 3286.1383 sec, Std Dev: 95.3766 sec, CV: 2.90%


-------------------------------

**----------Exhaustive Analysis------------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 16786.5100 sec, Std Dev: 262.6902',
    'Total Time - Mean: 18796.7650 sec, Std Dev: 56.2645',
    'Total Time - Mean: 21222.4500 sec, Std Dev: 18.9787'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 18935.2417 sec, Std Dev: 89.7726 sec, CV: 0.47%


---------------------

**----------VP Log Threshold Analysis-----------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 174.4664 sec, Std Dev: 0.8444',
    'Total Time - Mean: 169.7877 sec, Std Dev: 1.0580',
    'Total Time - Mean: 166.9165 sec, Std Dev: 1.3378'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 170.3902 sec, Std Dev: 0.6344 sec, CV: 0.37%


---

**-----------VP Evo Alg-------------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 162.0092 sec, Std Dev: 2.2646',
    'Total Time - Mean: 160.3141 sec, Std Dev: 1.1241',
    'Total Time - Mean: 165.9798 sec, Std Dev: 1.2691'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 162.7677 sec, Std Dev: 0.9430 sec, CV: 0.58%


--------------------

### Comparison between static-log and lamarckian
---

In [ ]:
import re
import math
from typing import Tuple, Optional

def parse_stats(s: str) -> Tuple[float, float]:
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    mean = float(mean_match.group(1))
    std  = float(std_match.group(1))
    return mean, std


def compare_policies(static_stats: str,
                     lamarckian_stats: str,
                     n_static: Optional[int] = None,
                     n_lam: Optional[int] = None):
    """
    Keeps the original outputs, and adds delta_se if run counts are provided.

    delta_std = sqrt(sd_s^2 + sd_l^2)  (SD of difference across random runs)
    delta_se  = sqrt(sd_s^2/n_s + sd_l^2/n_l)  (SE of mean difference)
    """
    mu_s, sigma_s = parse_stats(static_stats)
    mu_l, sigma_l = parse_stats(lamarckian_stats)

    delta = mu_s - mu_l
    sigma_delta = math.sqrt(sigma_s**2 + sigma_l**2)
    normalized = delta / mu_s if mu_s != 0 else float("nan")
    z_score = delta / sigma_delta if sigma_delta > 0 else float("inf")

    out = {
        "delta_mean": delta,
        "delta_std": sigma_delta,
        "normalized_improvement": normalized,
        "z_score": z_score,
    }

    if n_static is not None and n_lam is not None and n_static > 0 and n_lam > 0:
        delta_se = math.sqrt((sigma_s**2)/n_static + (sigma_l**2)/n_lam)
        out["delta_se"] = delta_se

    return out

---

In [ ]:
static = "Combined Mean: 170.3902 sec, Std Dev: 0.6344"
lam    = "Combined Mean: 162.7677 sec, Std Dev: 0.9430"

res = compare_policies(static, lam, n_static=3, n_lam=3)
for k, v in res.items():
    print(f"{k}: {v:.4f}")

delta_mean: 7.6225
delta_std: 1.1365
normalized_improvement: 0.0447
z_score: 6.7068
delta_se: 0.6562


---

### Speedup Analysis
---

In [ ]:
import re
import math
from typing import Dict, Tuple

def parse_time_stats(s: str) -> Tuple[float, float]:
    """
    Parse strings like:
    'Combined Mean: 29.5141 sec, Std Dev: 0.7580'
    Returns (mean, std_dev)
    """
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    return float(mean_match.group(1)), float(std_match.group(1))


def compute_speedups(
    stats: Dict[str, str],
    runs: Dict[str, int],
    baseline: str = "Exhaustive",
):
    """
    stats: dict of method -> stats string
    runs:  dict of method -> number of runs
    baseline: method name used as denominator

    Prints speedup S_i = T_baseline / T_i and SE(S_i).
    """
    if baseline not in stats:
        raise ValueError("Baseline method not found in stats.")

    # Parse baseline
    T_exh, sd_exh = parse_time_stats(stats[baseline])
    n_exh = runs[baseline]
    se_exh = sd_exh / math.sqrt(n_exh)

    print(f"Baseline: {baseline}")
    print("-" * 60)
    print(f"{'Method':<15} {'S_i':>10} {'SE(S_i)':>12}")
    print("-" * 60)

    for method, s in stats.items():
        if method == baseline:
            print(f"{method:<15} {1.0:>10.3f} {'-':>12}")
            continue

        T_i, sd_i = parse_time_stats(s)
        n_i = runs[method]
        se_i = sd_i / math.sqrt(n_i)

        # Speedup
        S = T_exh / T_i

        # SE via delta method
        se_S = S * math.sqrt(
            (se_exh / T_exh) ** 2 +
            (se_i   / T_i)   ** 2
        )

        print(f"{method:<15} {S:>10.3f} {se_S:>12.3f}")

---

In [ ]:
stats = {
    "VP_MWV":     "Combined Mean: 3286.1383 sec, Std Dev: 95.3766",
    "Exhaustive": "Combined Mean: 18935.2417 sec, Std Dev: 89.7726",
    "Log":        "Combined Mean: 170.3902 sec, Std Dev: 0.6344",
    "Lamarckian": "Combined Mean: 162.7677 sec, Std Dev: 0.9430",
}

runs = {
    "VP_MWV": 3,
    "Exhaustive": 3,
    "Log": 3,
    "Lamarckian": 3,
}

compute_speedups(stats, runs)

Baseline: Exhaustive
------------------------------------------------------------
Method                 S_i      SE(S_i)
------------------------------------------------------------
VP_MWV               5.762        0.098
Exhaustive           1.000            -
Log                111.129        0.387
Lamarckian         116.333        0.503
